In [14]:
import io
import os
import glob
# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types
import csv
from  cv2 import *
import numpy as np
import scipy
from PIL import Image
import pytesseract
import argparse
import os
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'
from PIL import Image, ImageEnhance, ImageFilter
import PyPDF2
from pdf2image import convert_from_path
from __future__ import print_function
from wand.image import Image
import imutils
%matplotlib inline
from matplotlib import pyplot as plt
import json
import re
from passporteye.mrz.image import MRZPipeline

In [12]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/dsquad/PythonNotebooks/try9.json"
#print(os.environ['GOOGLE_APPLICATION_CREDENTIALS']) 
# Instantiates a client
client = vision.ImageAnnotatorClient()
count = 0

In [15]:
def process(img):
    try:
        Imagesize=''
        img = cv2.imread(img)
        h,w,c= img.shape
        #print('firststep')
        #plt.figure()
        # Creating a Named window to display image
        #plt.imshow(img)
        # Display image
        #print(img.shape)
        img = cv2.resize(img, (560, 400), interpolation = cv2.INTER_CUBIC)
        #print(img.shape)
        #plt.imshow(img)

        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        #plt.imshow(img_gray)

        noise_removal = cv2.bilateralFilter(img_gray, 9, 75, 75)
        #plt.imshow(noise_removal)

        equal_histogram = cv2.equalizeHist(noise_removal)
        #plt.imshow(equal_histogram)

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        morph_image = cv2.morphologyEx(equal_histogram, cv2.MORPH_OPEN, kernel, iterations = 15)
        #plt.imshow(morph_image)

        sub_morp_image = cv2.subtract(equal_histogram, morph_image)
        #plt.imshow(sub_morp_image)

        ret, thresh_image = cv2.threshold(sub_morp_image, 100, 255, cv2.THRESH_OTSU)
        #plt.imshow(thresh_image)

        canny_image = cv2.Canny(thresh_image, 250, 255)
        canny_image = cv2.convertScaleAbs(canny_image)
        #plt.imshow(canny_image)

        kernel = np.ones((3, 3), np.uint8)
        # Creating the kernel for dilation
        dilated_image = cv2.dilate(canny_image, kernel, iterations = 1)
        #plt.imshow(dilated_image)

        new, contours, hierarchy = cv2.findContours(dilated_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key = cv2.contourArea, reverse = True)[: 10]
        # Sort the contours based on area, so that the number plate will be in top 10 contours
        screenCnt = None
        # loop over our contours
        #plt.ion()

        for c in contours:
        # approximate the contour
            peri = cv2.arcLength(c, True)
        #print(peri)
            approx = cv2.approxPolyDP(c, 0.06 * peri, True)
        #print(approx) # Approximating with 6 % error
        # if our approximated contour has four points, then
        # we can assume that we have found our screen
        #print(len(approx))
            if len(approx) == 4: # Select the contour with 4 corners
                screenCnt = approx
        #else :
        #print('contours problem')
        #Status = 'No Contours detected'
            break
        final = cv2.drawContours(img, [screenCnt], -1, (0, 255, 0), 3)
        #imgplot = plt.imshow(final)
        #plt.show(block = False)
        #plt.pause(3)
        #check = input("is image correct?")
        #plt.imshow(final)
        mask = np.zeros(img_gray.shape, np.uint8)
        new_image = cv2.drawContours(mask, [screenCnt], 0, 255, -1, )
        new_image = cv2.bitwise_and(img, img, mask = mask)
        # Histogram equal for enhancing the number plate for further processing
        y, cr, cb = cv2.split(cv2.cvtColor(new_image, cv2.COLOR_RGB2YCrCb))
        # Converting the image to YCrCb model and splitting the 3 channels
        y = cv2.equalizeHist(y)
        # Applying histogram equalisation
        final_image = cv2.cvtColor(cv2.merge([y, cr, cb]), cv2.COLOR_YCrCb2RGB)
        #plt.imshow(final_image)
        DetectedPlate_byCV2 = pytesseract.image_to_string(final_image)
        #cv2.imwrite('./carimages/f/hey.jpg', final_image)
        #img = cv2.imread('./carimages/f/hey.jpg')
        #imgBlurred = cv2.GaussianBlur(img, (5, 5), 0)
        #gray = cv2.cvtColor(imgBlurred, cv2.COLOR_BGR2GRAY)
        #ret2, threshold_img = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY_INV) #+cv2.THRESH_OTSU)
        #g = pytesseract.image_to_string(threshold_img)
        if len(DetectedPlate_byCV2) > 0 :
            Message_byCV2 = 'Plate successfully found by CV2' 
            Status_byCV2 = 'O'
            return Imagesize,Status_byCV2, DetectedPlate_byCV2, Message_byCV2,w,  h
        else:
            Message_byCV2 = 'No Plate found by CV2' 
            return Imagesize,Status_byCV2, DetectedPlate_byCV2, Message_byCV2,w, h
    except:
        Message_byCV2 = 'No Plate found by CV2'
        Status_byCV2='N'
        DetectedPlate_byCV2 =""
        return Imagesize,Status_byCV2, DetectedPlate_byCV2, Message_byCV2, w,h


In [17]:
with open("./carimages/LicensePlatesOCR4.csv", "w", encoding="utf-8") as LicenseFile:
    fieldnames = ['File_Name','Imagewidth','Imageheight','Status_byGoogle','Dectected_Plate_byGoogle','Message_byGoogle','Metadata', 'Status_byCV2','DetectedPlate_byCV2','Message_byCV2']
    writer = csv.DictWriter(LicenseFile, fieldnames=fieldnames)
    writer.writeheader()
    for img in glob.glob('./carimages/NewLicensePlates/*.jpg'):
        with open(img, 'rb') as image_file:
            Imagesize=''
            Status_byGoogle ='N'
            Message_byGoogle = ''
            Metadata= ''
            t=''
            s=''
            w=''
            Status_byCV2='N'
            DetectedPlate_byCV2 =""
            Message_byCV2=''
            content = image_file.read()
            image = vision.types.Image(content=content)
            response = client.document_text_detection(image=image)
            texts=(response.text_annotations)
            #i = cv2.imread(img)

            #print (texts)
            if len(texts) > 0 :
                strange=texts[0].description
                match1 = re.search('[A-Z][A-Z]([\s\.-]?)([\s\.-]?)([0-9][0-9][0-9])([\s\.-]?)([\s\.-]?)[A-Z][A-Z]', strange)
                match2 = re.search('[0-9][0-9]([0-9]?)([\s\.-]?)([\s\.-]?)([A-Z][A-Z][A-Z])([\s\.-]?)([\s\.-]?)[0-9][0-9]', strange)
                match3 = re.search('[0-9][0-9][0-9][0-9]([\s\.-]?)([\s\.-]?)([A-Z][A-Z])([\s\.-]?)([\s\.-]?)[0-9][0-9]', strange)                
                if match1:
                    #print ( "plate found", match1.group()) 
                    t=match1.group()
                    Status_byGoogle ='O'
                    Message_byGoogle= 'Plate successfully found by Google API'
                    Metadata= ''
                elif match2:
                    #print ( "plate found", match2.group()) 
                    t=match2.group()
                    Status_byGoogle ='O'
                    Message_byGoogle= 'Plate successfully found by Google API'
                    Metadata= ''
                elif match3:
                    #print ( "plate found", match2.group()) 
                    t=match3.group()
                    Status_byGoogle ='O'
                    Message_byGoogle= 'Plate successfully found by Google API'
                    Metadata= ''                    
                else:
                    Message_byGoogle ='No Plate found by Google API'
                    Metadata=  strange
            else:
                Message_byGoogle ='No Plate found by Google API'
            Imagesize,Status_byCV2, DetectedPlate_byCV2, Message_byCV2,  w,h=process(img)
            writer.writerow({'File_Name': img, 'Imagewidth':w,'Imageheight' :h ,'Status_byGoogle':Status_byGoogle , 'Dectected_Plate_byGoogle': t, 'Message_byGoogle': Message_byGoogle,'Metadata':Metadata,'Status_byCV2': Status_byCV2,'DetectedPlate_byCV2':DetectedPlate_byCV2,'Message_byCV2':Message_byCV2})  
  